In [1]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio.windows import Window
import math
import zarr

In [2]:
src = rasterio.open('/media/feromes/FEROMES/VHM/0-VHM-sao-paulo-city.tif')

In [3]:
profile = src.profile

In [4]:
upscale = 1
chunk_size = 20000
w_scaled = math.floor(src.width / upscale)
h_scaled = math.floor(src.height / upscale)
w_size = math.ceil(w_scaled/chunk_size)
h_size = math.ceil(h_scaled/chunk_size)

In [5]:
w_scaled, h_scaled

(94455, 140661)

In [6]:
w_size, h_size

(5, 8)

In [7]:
vhm_temp = zarr.open(
    '../tmp/bhm.zarr', 
    mode='w', 
    shape=(w_size*chunk_size, h_size*chunk_size), 
    chunks=(chunk_size,chunk_size), 
    dtype=np.float32
)

In [8]:
vhm = np.ones((src.count, chunk_size, chunk_size), dtype='float32')

for w in range(w_size):
    for h in range(h_size):
        print(w,h)
        
        
        w1 = (w+1)*chunk_size*upscale
        h1 = (h+1)*chunk_size*upscale

        # if w1 > w_scaled:
        #     w1 = src.width

        # if h1 > h_scaled:
        #     h1 = src.height
      
        data = src.read(
            # out_shape=(src.count, chunk_size, chunk_size),
            out=vhm,
            window=Window.from_slices(
                slice(w*chunk_size*upscale, w1), 
                slice(h*chunk_size*upscale, h1)
            ),
            # resampling=Resampling.average,
            boundless=True,
            fill_value=0.
        )

        vhm_temp[w*chunk_size:w1//upscale, h*chunk_size:h1//upscale] = vhm[0]
            

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
